In [20]:
import numpy as np
import pandas as pd
import scipy.sparse as ssp
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm
from lightgbm.sklearn import LGBMClassifier
import os 
import re
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 

# I will use cross_val_score on XGBoost to select 100,200 or 300 for each preprocess

In [2]:
work_train_w2v = {} 
work_test_w2v = {}
pre_process=["w2v_100.csv","w2v_200.csv","w2v_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_w2v[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test_w2v[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [3]:
work_train_bio = {} 
work_test_bio = {}
pre_process=["bio.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_bio[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test_bio[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [4]:
work_train_tfidf = {} 
work_test_tfidf = {}
pre_process=["tfidf_tsvd_100.csv","tfidf_tsvd_200.csv","tfidf_tsvd_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_tfidf[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test_tfidf[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [21]:
new_train=pd.read_csv("checkpoints_databases/new_working_train.csv")
new_test=pd.read_csv("checkpoints_databases/new_working_test.csv")

In [22]:
y=np.array(new_train.Class)-1

In [23]:
ID_train=new_train.ID
ID_test=new_test.ID


In [24]:
kf = model_selection.StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

Start 

In [33]:
clf_xgb=XGBClassifier(max_depth=5, objective="multi:softprob",seed=26)
for name in work_train_w2v:
    h=cross_val_score(clf_xgb,np.array(work_train_w2v[name]),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))
#300

meanw2v_100 -1.07331330963 std:w2v_100 0.0364550094376
meanw2v_200 -1.08126323561 std:w2v_200 0.0447141389549
meanw2v_300 -1.06656954989 std:w2v_300 0.0304045282825


In [34]:
for name in work_train_bio:
    h=cross_val_score(clf_xgb,np.array(work_train_bio[name]),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))
#k

meanbio -1.09764345161 std:bio 0.034944053108


In [35]:
for name in work_train_tfidf:
    h=cross_val_score(clf_xgb,np.array(work_train_tfidf[name]),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))
#100 tsvd

meantfidf_tsvd_100 -1.00962632533 std:tfidf_tsvd_100 0.0343409770402
meantfidf_tsvd_200 -1.01784942814 std:tfidf_tsvd_200 0.0316089971765
meantfidf_tsvd_300 -1.01995911039 std:tfidf_tsvd_300 0.0456225357959


# GRID SEARCH PHASE ALL ALGOS

In [7]:
work_train= {} 
work_test = {}
#To complete
pre_process=["bio.csv","tfidf_tsvd_100.csv","w2v_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [31]:
feat_train=pd.read_csv("../window/w_meta_features/meta_train_l1l2.csv")
feat_test=pd.read_csv("../window/w_meta_features/meta_test_l1l2.csv")
feat_train=feat_train.drop("ID",axis=1)
feat_test=feat_test.drop("ID",axis=1)
feat_ext_train=pd.read_csv("w_meta_features/new_working_train_ext.csv")
feat_ext_test=pd.read_csv("w_meta_features/new_working_test_ext.csv")

In [15]:
#add a "external model" xgb alone

In [15]:
ext_bio=pd.read_csv("../l2_meta_features/svd25_biological.csv")
ext_cell=pd.read_csv("../l2_meta_features/svd25_cellular.csv")
ext_mol=pd.read_csv("../l2_meta_features/svd25_molecular.csv")

In [16]:
ext_bio=ext_bio.drop(["Class","Gene","ID","Variation"],axis=1)
ext_cell=ext_cell.drop(["Class","Gene","ID","Variation"],axis=1)
ext_mol=ext_mol.drop(["Class","Gene","ID","Variation"],axis=1)

In [17]:
model_ext=pd.concat((ext_bio,ext_cell,ext_mol),axis=1)
model_ext_train=model_ext[:len(y)]
model_ext_test=model_ext[len(y):]

In [ ]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3],
    "n_estimators" : [100,200],
    "subsample":[0.8,1],
    "colsample_bytree":[0.8,1]
}

gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
gsearch.fit(np.array(model_ext_train),y)
print(name)
print(gsearch.grid_scores_)
print(gsearch.best_params_)
print(gsearch.best_score_)
# best grid : {'colsample_bytree': 1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 21.3min finished


bio
[mean: -1.16119, std: 0.03745, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}, mean: -1.16675, std: 0.04054, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1}, mean: -1.18082, std: 0.04592, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.8}, mean: -1.17657, std: 0.04654, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 1}, mean: -1.16237, std: 0.03865, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}, mean: -1.16553, std: 0.04232, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1}, mean: -1.17751, std: 0.04598, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 0.8}, 

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [2]:
#now add the others models that worked on the window pre-processed

In [33]:
#Gene features + manual features on variations + text embeddings
work_train_final={}
work_test_final={}
work_train_ext={}
work_test_ext={}
for name in work_train:
    work_train_final[name]=pd.concat((work_train[name],feat_train),axis=1)
    work_test_final[name]=pd.concat((work_test[name],feat_test),axis=1)
    work_train_ext[name]=pd.concat((work_train[name],feat_ext_train),axis=1)
    work_test_ext[name]=pd.concat((work_test[name],feat_ext_test),axis=1)

In [ ]:
work_train_franck=pd.concat((work_train["tfidf_tsvd_100"],))

In [19]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3,5,7],
    "n_estimators" : [100,200],
    "subsample":[0.8,0.9,1],
    "colsample_bytree":[0.8,0.9,1]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#bio : {'colsample_bytree': 0.8, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1}
#tfidf: {'colsample_bytree': 0.8, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1}
#w2v: {'colsample_bytree': 0.8, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 37.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 98.6min


KeyboardInterrupt: 

In [ ]:
clf_lgbm=LGBMClassifier(seed=26)
param_test= {
    'n_estimators': [8,,16,24,36,48],
    'num_leaves': [6,12,16,22],
    'boosting_type' : ['gbdt'],
    'colsample_bytree' : [0.7,0.8,0.9],
    'subsample' : [0.7,0.8,0.9]
    }
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_lgbm, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v : {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.8}
#tfidf : {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.8}
#w2v : {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.8}

In [ ]:
clf_log=LogisticRegression()
param_test= {
    "C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty" : ["l1","l2"]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_log, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#bio : {'C': 10, 'penalty': 'l1'}
#tfidf : {'C': 10, 'penalty': 'l1'}
#w2v : {'C': 1, 'penalty': 'l1'}

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  5.5min finished


bio
[mean: -2.16489, std: 0.00029, params: {'C': 0.001, 'penalty': 'l1'}, mean: -2.10134, std: 0.00074, params: {'C': 0.001, 'penalty': 'l2'}, mean: -1.91341, std: 0.00133, params: {'C': 0.01, 'penalty': 'l1'}, mean: -1.87990, std: 0.00458, params: {'C': 0.01, 'penalty': 'l2'}, mean: -1.74917, std: 0.01223, params: {'C': 0.1, 'penalty': 'l1'}, mean: -1.60313, std: 0.01831, params: {'C': 0.1, 'penalty': 'l2'}, mean: -1.36559, std: 0.02814, params: {'C': 1, 'penalty': 'l1'}, mean: -1.36289, std: 0.02980, params: {'C': 1, 'penalty': 'l2'}, mean: -1.16996, std: 0.04263, params: {'C': 10, 'penalty': 'l1'}, mean: -1.20232, std: 0.03268, params: {'C': 10, 'penalty': 'l2'}, mean: -1.38548, std: 0.06647, params: {'C': 100, 'penalty': 'l1'}, mean: -1.16012, std: 0.04500, params: {'C': 100, 'penalty': 'l2'}, mean: -1.72748, std: 0.12829, params: {'C': 1000, 'penalty': 'l1'}, mean: -1.30786, std: 0.05802, params: {'C': 1000, 'penalty': 'l2'}]
{'C': 100, 'penalty': 'l2'}
-1.16011995766
Fitting 5 fo

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   40.0s finished


tfidf_tsvd_100
[mean: -2.16489, std: 0.00029, params: {'C': 0.001, 'penalty': 'l1'}, mean: -2.12527, std: 0.00046, params: {'C': 0.001, 'penalty': 'l2'}, mean: -1.91341, std: 0.00133, params: {'C': 0.01, 'penalty': 'l1'}, mean: -1.89633, std: 0.00310, params: {'C': 0.01, 'penalty': 'l2'}, mean: -1.58242, std: 0.01423, params: {'C': 0.1, 'penalty': 'l1'}, mean: -1.52553, std: 0.01136, params: {'C': 0.1, 'penalty': 'l2'}, mean: -1.16043, std: 0.01439, params: {'C': 1, 'penalty': 'l1'}, mean: -1.19221, std: 0.01642, params: {'C': 1, 'penalty': 'l2'}, mean: -1.08386, std: 0.02675, params: {'C': 10, 'penalty': 'l1'}, mean: -1.07288, std: 0.02133, params: {'C': 10, 'penalty': 'l2'}, mean: -1.20319, std: 0.04193, params: {'C': 100, 'penalty': 'l1'}, mean: -1.10435, std: 0.03337, params: {'C': 100, 'penalty': 'l2'}, mean: -1.30641, std: 0.06715, params: {'C': 1000, 'penalty': 'l1'}, mean: -1.19175, std: 0.04642, params: {'C': 1000, 'penalty': 'l2'}]
{'C': 10, 'penalty': 'l2'}
-1.07288442852
Fi

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   39.1s


In [ ]:
clf_log=LogisticRegression()
param_test= {
    "C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty" : ["l1","l2"]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_log, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_ext[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)

In [ ]:
clf_ada=AdaBoostClassifier(n_estimators=100, learning_rate=0.3, algorithm="SAMME.R", random_state=26)
param_test={
    "n_estimators":[50,70,100,200],
    "learning_rate":[0.01,0.05,0.1,0.2]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_ada, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v : {'learning_rate': 0.01, 'n_estimators': 50}
#tfidf : {'learning_rate': 0.01, 'n_estimators': 70}
#w2v :{'learning_rate': 0.01, 'n_estimators': 50}

In [ ]:
clf_dt=RandomForestClassifier()
param_test={
    "max_depth":[10,15,20,25,30,35],
    "n_estimators":[200,300,400]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_dt, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v : {'max_depth': 15, 'n_estimators': 300}
#tfidf :{'max_depth': 20, 'n_estimators': 400}
#w2v : {'max_depth': 15, 'n_estimators': 400}

# TRAINING PHASE ALL ALGOS 

In [23]:
def model_gen(X,X_test,y,classifier,file,five_fold_predict=True):
    #if not os.path.exists("scores/"+file):
    #   os.makedirs("scores/"+file)
    if five_fold_predict:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
        
            fold += 1

            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)

            clf=classifier
            clf.fit(X_train,y_train)
            p_test = clf.predict_proba(X_test)
            y_test += p_test/5

    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("new_scores/new_stack_test/new_{}.csv".format(file),index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb

    y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
    subm1 = pd.DataFrame(y_pred, columns=classes)
    subm1['ID'] = ID_train
    subm1.to_csv("new_scores/new_stack_train/new_{}.csv".format(file),index=False)


In [138]:
dic_xgb={"xgb_bio":XGBClassifier(colsample_bytree=0.8,max_depth=7,min_child_weight=3,n_estimators=100,subsample=0.8,objective="multi:softprob",seed=26),
        "xgb_tfidf":XGBClassifier(colsample_bytree=0.8,max_depth=5,min_child_weight=3,n_estimators=100,subsample=1,objective="multi:softprob",seed=26),
        "xgb_w2v":XGBClassifier(colsample_bytree=0.8,max_depth=5,min_child_weight=3,n_estimators=100,subsample=0.8,objective="multi:softprob",seed=26)}

dic_lgbm={"lgbm_bio":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26),
        "lgbm_tfidf":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26),
        "lgbm_w2v":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26)}
dic_lr={"lr_bio":LogisticRegression(C=10,penalty="l1"),
        "lr_tfidf":LogisticRegression(C=10,penalty="l1"),
        "lr_w2v":LogisticRegression(C=1,penalty="l1")}
dic_ada={"ada_bio":AdaBoostClassifier(n_estimators=50, learning_rate=0.01, algorithm="SAMME.R", random_state=26),
        "ada_tfidf":AdaBoostClassifier(n_estimators=70, learning_rate=0.01, algorithm="SAMME.R", random_state=26),
        "ada_w2v":AdaBoostClassifier(n_estimators=50, learning_rate=0.01, algorithm="SAMME.R", random_state=26)}
dic_rf={"rf_bio":RandomForestClassifier(n_estimators=400,max_depth=25,random_state=26),
        "rf_tfidf":RandomForestClassifier(n_estimators=400,max_depth=20,random_state=26),
        "rf_w2v":RandomForestClassifier(n_estimators=300,max_depth=15,random_state=26)}

print("xgboost here")
for clf,name in zip(dic_xgb.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name].drop("ID",axis=1)),y=y,classifier=dic_xgb[clf],file=clf)
print("lgbm here")
for clf,name in zip(dic_lgbm.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name].drop("ID",axis=1)),y=y,classifier=dic_lgbm[clf],file=clf)
print("logreg here")
for clf,name in zip(dic_lr.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name].drop("ID",axis=1)),y=y,classifier=dic_lr[clf],file=clf)
print("adaboost here")
for clf,name in zip(dic_ada.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name].drop("ID",axis=1)),y=y,classifier=dic_ada[clf],file=clf)
print("random forest here")
for clf,name in zip(dic_rf.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name].drop("ID",axis=1)),y=y,classifier=dic_rf[clf],file=clf)

ext_xgb=XGBClassifier(tocomplete)
model_gen(X=np.array(model_ext_train),X_test=np.array(model_ext_test),y=y,classifier=ext_xgb,file="xgb_external")

xgboost here
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 243) (743, 243)
Fold 2 (2949, 243) (740, 243)
Fold 3 (2952, 243) (737, 243)
Fold 4 (2954, 243) (735, 243)
Fold 5 (2955, 243) (734, 243)
cross_val sur train 
lgbm here
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 243) (743, 243)
Fold 2 (2949, 243) (740, 243)
Fold 3 (2952, 243) (737, 243)
Fold 4 (2954, 243) (735, 243)
Fo

In [ ]:
ex